In [1]:
# 다음의 코드를 참고하여 재구성 : https://github.com/gaborvecsei/CDCGAN-Keras/tree/master/cdcgan 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
cd /content/drive/My Drive/Colab Notebooks/Paper/temp/mnist_GAN

/content/drive/My Drive/Colab Notebooks/Paper/temp/mnist_GAN


In [4]:
import matplotlib

matplotlib.use('Agg')
import numpy as np
from tensorflow.keras import backend as K
from tensorflow.keras import utils
from tensorflow.keras import optimizers
from tensorflow.keras import datasets
from tqdm import tqdm
import cdcgan_models, cdcgan_utils
import os

BATCH_SIZE = 128
EPOCHS = 100

In [5]:
# Load & Prepare MNIST

(X_train, y_train), (_, _) = datasets.mnist.load_data()
X_train = cdcgan_utils.transform_images(X_train)
X_train = X_train[:, :, :, None]

y_train = utils.to_categorical(y_train, 100)

# Create the models

print("Generator:")
G = cdcgan_models.generator_model()
G.summary()


11493376/11490434 [==============================] - 0s 0us/step
Generator:
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 1024)         103424      input_1[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 1024)         103424      input_2[0][0]                    
__________________

In [6]:
print("Discriminator:")
D = cdcgan_models.discriminator_model()
D.summary()

Discriminator:
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 28, 28, 64)   1088        input_3[0][0]                    
__________________________________________________________________________________________________
activation (Activation)         multiple             0           conv2d_2[0][0]                   
                                                                 conv2d_3[0][0]                   
                                                                 dense_4[0][0]                    
                                                                 dense_6[0][0

In [7]:
print("Combined:")
GD = cdcgan_models.generator_containing_discriminator(G, D)
GD.summary()

Combined:
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
model (Model)                   (None, 28, 28, 1)    13377857    input_5[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
model_1 (Model)                 (None, 1)            10084161    model[1][0]      

In [8]:
optimizer = optimizers.Adam(0.0002, 0.5)

G.compile(loss='binary_crossentropy', optimizer=optimizer,metrics=['accuracy'])
GD.compile(loss='binary_crossentropy', optimizer=optimizer,metrics=['accuracy'])
D.trainable = True
D.compile(loss='binary_crossentropy', optimizer=optimizer,metrics=['accuracy'])

In [9]:
# Setup Tensorboard loggers

# tfboard_loggers.TFBoardModelGraphLogger.log_graph("../models/logs", K.get_session())
# loss_logger = tfboard_loggers.TFBoardScalarLogger("../models/logs/loss")
# image_logger = tfboard_loggers.TFBoardImageLogger("../models/logs/generated_images")

In [10]:
# Model Training

iteration = 0

nb_of_iterations_per_epoch = int(X_train.shape[0] / BATCH_SIZE)
print("Number of iterations per epoch: {0}".format(nb_of_iterations_per_epoch))

for epoch in range(EPOCHS):
    pbar = tqdm(desc="Epoch: {0}".format(epoch), total=X_train.shape[0])

    g_losses_for_epoch = []
    d_losses_for_epoch = []
    g_accuracies_for_epoch = []
    d_accuracies_for_epoch = [] 

    for i in range(nb_of_iterations_per_epoch):
        noise = cdcgan_utils.generate_noise((BATCH_SIZE, 100))

        image_batch = X_train[i * BATCH_SIZE:(i + 1) * BATCH_SIZE]
        label_batch = y_train[i * BATCH_SIZE:(i + 1) * BATCH_SIZE]

        generated_images = G.predict([noise, label_batch], verbose=0)

        if i % 20 == 0:
            image_grid = cdcgan_utils.generate_mnist_image_grid(G,
                                                                title="Epoch {0}, iteration {1}".format(epoch,
                                                                                                        iteration))
            cdcgan_utils.save_generated_image(image_grid, epoch, i, "/content/drive/My Drive/Colab Notebooks/Paper/temp/mnist_GAN/images/generated_mnist_images_per_iteration")
            #image_logger.log_images("generated_mnist_images_per_iteration", [image_grid], iteration)

        X = np.concatenate((image_batch, generated_images))
        y = np.array([1] * BATCH_SIZE + [0] * BATCH_SIZE)
        label_batches_for_discriminator = np.concatenate((label_batch, label_batch))

        D_loss,D_acc = D.train_on_batch([X, label_batches_for_discriminator], y)
        d_losses_for_epoch.append(D_loss)
        d_accuracies_for_epoch.append(D_acc)
        #loss_logger.log_scalar("discriminator_loss", D_loss, iteration)

        noise = cdcgan_utils.generate_noise((BATCH_SIZE, 100))
        D.trainable = False
        G_loss,G_acc = GD.train_on_batch([noise, label_batch], np.array([1] * BATCH_SIZE))
        D.trainable = True
        g_losses_for_epoch.append(G_loss)
        g_accuracies_for_epoch.append(G_acc)
        #loss_logger.log_scalar("generator_loss", G_loss, iteration)

        pbar.update(BATCH_SIZE)

        iteration += 1

    # Save a generated image for every epoch
    image_grid = cdcgan_utils.generate_mnist_image_grid(G, title="Epoch {0}".format(epoch))
    cdcgan_utils.save_generated_image(image_grid, epoch, 0, "/content/drive/My Drive/Colab Notebooks/Paper/temp/mnist_GAN/images/generated_mnist_images_per_epoch")
    #image_logger.log_images("generated_mnist_images_per_epoch", [image_grid], epoch)

    pbar.close()
    print("D loss: {0}, GD loss: {1} // D accuracy: {2}, GD accuracy: {3}".format(np.mean(d_losses_for_epoch), np.mean(g_losses_for_epoch),np.mean(d_accuracies_for_epoch), np.mean(g_accuracies_for_epoch)))

    G.save_weights(os.path.join("/content/drive/My Drive/Colab Notebooks/Paper/temp/mnist_GAN/G", str(epoch)+".h5"))
    D.save_weights(os.path.join("/content/drive/My Drive/Colab Notebooks/Paper/temp/mnist_GAN/D", str(epoch)+".h5"))

Epoch: 0:   0%|          | 0/60000 [00:00<?, ?it/s]

Number of iterations per epoch: 468


Epoch: 0: 100%|█████████▉| 59904/60000 [01:07<00:00, 893.17it/s] 


D loss: 0.09624708039346166, GD loss: 0.338146458344695 // D accuracy: 0.9814369658119658, GD accuracy: 0.9138287927350427


Epoch: 1: 100%|█████████▉| 59904/60000 [00:59<00:00, 1013.75it/s]


...

D loss: 0.5317218397290279, GD loss: 1.1232861549171627 // D accuracy: 0.7413695245726496, GD accuracy: 0.20638020833333334


Epoch: 3: 100%|█████████▉| 59904/60000 [00:57<00:00, 1044.54it/s]


D loss: 0.5836410435855898, GD loss: 0.9863044223469547 // D accuracy: 0.6863064236111112, GD accuracy: 0.2625033386752137


Epoch: 4:  26%|██▋       | 15872/60000 [00:15<00:47, 938.13it/s]

KeyboardInterrupt: ignored

In [ ]:
os.path.join("../G", str(epoch)+".h5")